In [1]:
from pyspark.sql.session import SparkSession

spark = SparkSession\
    .builder\
    .master("local[*]")\
    .appName("strata")\
    .getOrCreate()

google_gmail_emails = spark\
    .read\
    .format("csv")\
    .option("header",True)\
    .option("inferSchema",True)\
    .option("delimiter", "\t")\
    .load("/Users/sahilnagpal/Desktop/finding-sql/stratascratch/data/google_gmail_emails.csv")

google_gmail_emails.show()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/27 15:41:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/07/27 15:41:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


+---+------------------+------------------+---+
| id|         from_user|           to_user|day|
+---+------------------+------------------+---+
|  0|6edf0be4b2267df1fa|75d295377a46f83236| 10|
|  1|6edf0be4b2267df1fa| 32ded68d89443e808|  6|
|  2|6edf0be4b2267df1fa|55e60cfcc9dc49c17e| 10|
|  3|6edf0be4b2267df1fa|e0e0defbb9ec47f6f7|  6|
|  4|6edf0be4b2267df1fa|47be2887786891367e|  1|
|  5|6edf0be4b2267df1fa|2813e59cf6c1ff698e|  6|
|  6|6edf0be4b2267df1fa|a84065b7933ad01019|  8|
|  7|6edf0be4b2267df1fa|850badf89ed8f06854|  1|
|  8|6edf0be4b2267df1fa|6b503743a13d778200|  1|
|  9|6edf0be4b2267df1fa|d63386c884aeb9f71d|  3|
| 10|6edf0be4b2267df1fa|5b8754928306a18b68|  2|
| 11|6edf0be4b2267df1fa|6edf0be4b2267df1fa|  8|
| 12|6edf0be4b2267df1fa|406539987dd9b679c0|  9|
| 13|6edf0be4b2267df1fa|114bafadff2d882864|  5|
| 14|6edf0be4b2267df1fa|157e3e9278e32aba3e|  2|
| 15|75d295377a46f83236|75d295377a46f83236|  6|
| 16|75d295377a46f83236|d63386c884aeb9f71d|  8|
| 17|75d295377a46f83236|55e60cfcc9dc49c1

In [4]:
from pyspark.sql.functions import col,dense_rank,count
from pyspark.sql.window import Window

calc_df = google_gmail_emails\
    .groupBy(col("from_user"))\
    .agg(count("id").alias("cnt"))

calc_df\
    .withColumn("row_number",dense_rank().over(Window.orderBy(col("cnt").desc(),col("from_user"))))\
    .show()

23/07/27 15:44:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/07/27 15:44:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/07/27 15:44:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/07/27 15:44:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------------------+---+----------+
|         from_user|cnt|row_number|
+------------------+---+----------+
| 32ded68d89443e808| 19|         1|
|ef5fe98c6b9f313075| 19|         2|
|5b8754928306a18b68| 18|         3|
|55e60cfcc9dc49c17e| 16|         4|
|91f59516cb9dee1e88| 16|         5|
|6edf0be4b2267df1fa| 15|         6|
|7cfe354d9a64bf8173| 15|         7|
| cbc4bd40cd1687754| 15|         8|
|e0e0defbb9ec47f6f7| 15|         9|
|8bba390b53976da0cd| 14|        10|
|a84065b7933ad01019| 14|        11|
|e22d2eabc2d4c19688| 13|        12|
|850badf89ed8f06854| 12|        13|
|aa0bd72b729fab6e9e| 12|        14|
|2813e59cf6c1ff698e| 11|        15|
|5dc768b2f067c56f77| 11|        16|
|5eff3a5bfc0687351e| 11|        17|
|75d295377a46f83236| 11|        18|
|157e3e9278e32aba3e| 10|        19|
|d63386c884aeb9f71d| 10|        20|
+------------------+---+----------+
only showing top 20 rows

